# Notebook Imports


In [25]:
import sys
import pandas as pd
from os import walk
from os.path import join

# Constants


In [40]:
EXAMPLE_FILE = 'SpamData/01_Processing/practice_email.txt'

SPAM_1_PATH = 'SpamData/01_Processing/spam_assassin_corpus/spam_1'
SPAM_2_PATH = 'SpamData/01_Processing/spam_assassin_corpus/spam_2'
EASY_NONSPAM_1_PATH = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_1'
EASY_NONSPAM_2_PATH = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_2'

SPAM_CAT = 1 ## spam classification
HAM_CAT = 0 ## ham is non spam classification

# Reading File

In [27]:
stream = open(EXAMPLE_FILE, encoding='latin-1')
message = stream.read()
stream.close()

print(type (message))
print(message)


<class 'str'>
From exmh-workers-admin@redhat.com  Thu Aug 22 12:36:23 2002
Return-Path: <exmh-workers-admin@spamassassin.taint.org>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36
	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:36:16 +0100 (IST)
Received: from listman.spamassassin.taint.org (listman.spamassassin.taint.org [66.187.233.211]) by
    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MBYrZ04811 for
    <zzzz-exmh@spamassassin.taint.org>; Thu, 22 Aug 2002 12:34:53 +0100
Received: from listman.spamassassin.taint.org (localhost.localdomain [127.0.0.1]) by
    listman.redhat.com (Postfix) with ESMTP id 8386540858; Thu, 22 Aug 2002
    07:35:02 -0400 (EDT)
Delivered-To: exmh-workers@listman.spamassassin.taint.org
Received: fro

In [28]:
sys.getfilesystemencoding()

'utf-8'

In [29]:
stream = open(EXAMPLE_FILE, encoding='latin-1')

is_body = False
lines = []

# line comes from a property of string, basically str with /n is a line
for line in stream:
    if is_body:
        lines.append(line)
    elif line == '\n':
        is_body = True

stream.close()

email_body = '\n'.join(lines)


print(email_body)



Dear Mr Still



Good tidings to you and all your staff for the festive season ahead (Christmas).

Now to the crux of the matter-in-hand: I am a fully qualified Santa Claus and am wondering whether you might consider me to run my own "Santa's Grotto" in your store.

But WAIT! You're probably thinking: "What makes him so special?"

Well, first of all, I have made several changes to the characterisation of Father Christmas. Rather than greeting the children with shouts of "Ho, ho, ho!" I prefer to whisper the phrase "Dependence is not unfathomable in this cruel world we live in". In addition, my gifts are ALL hand-made, ranging from felt hoops to vanilla-pod holders.

You will note also, from the enclosed sketch, that I have radically redesigned Santa's outfit and have renamed my character "Lord Buckles". Would you be interested in employing me? I promise NEVER to let you down.

I look forward to hearing from you.



Best wishes

Robin Cooper

[Excerpt from the book: The Timewaster Let

# Generator Functions

In [30]:
def generate_squares(N):
    for my_number in range(N):
        yield my_number **2

In [31]:
for i in generate_squares(3):
    print(i, end=' ->')

0 ->1 ->4 ->

## Email body extraction

In [32]:
def email_body_generator(path):
    
    for root, dirnames, filenames in walk(path):
        for file_name in filenames:
            
            filepath = join(root, file_name)

            stream = open(filepath, encoding='latin-1')
            
            is_body = False
            lines = []

            for line in stream:
                if is_body:
                    lines.append(line)
                elif line == '\n':
                    is_body = True

            stream.close()

            email_body = '\n'.join(lines)


            yield file_name, email_body

In [33]:
def df_from_directory(path, classification):
    rows = []
    row_names = []
    
    for file_name, email_body in email_body_generator(path):
        rows.append({'MESSAGE': email_body, 'CATEGORY': classification})
        row_names.append(file_name)
        
    return pd.DataFrame(rows, index=row_names)

In [38]:
spam_emails = df_from_directory(SPAM_1_PATH, 1)
spam_emails = spam_emails.append(df_from_directory(SPAM_2_PATH, 2))
spam_emails.head()

,MESSAGE,CATEGORY
00467.5b733c506b7165424a0d4a298e67970f,This is a multipart MIME message.\n\n\n\n--= M...,1
00031.a78bb452b3a7376202b5e62a81530449,"Suppliers of Computers, Printers, etc. & Consu...",1
00109.eda1664dd3b3c31b67e5cd04553b6546,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",1
00209.5276f967533f2ce0209c1eff631a86ff,This is a multi-part message in MIME format.\n...,1
00268.9b64189b6da55d1e0a30026ca73eb0da,HTML>\n\n<HEAD>\n\n<TITLE></TITLE>\n\n</HEAD>\...,1


In [39]:
spam_emails.shape


(1898, 2)

In [41]:
ham_emails = df_from_directory(EASY_NONSPAM_1_PATH, HAM_CAT)
ham_emails = ham_emails.append(df_from_directory(EASY_NONSPAM_2_PATH, HAM_CAT))
ham_emails.shape

(3901, 2)

In [42]:
data = pd.concat([spam_emails, ham_emails])
print('Shape of entire dataframe is', data.shape)
data.head()

Shape of entire dataframe is (5799, 2)


,MESSAGE,CATEGORY
00467.5b733c506b7165424a0d4a298e67970f,This is a multipart MIME message.\n\n\n\n--= M...,1
00031.a78bb452b3a7376202b5e62a81530449,"Suppliers of Computers, Printers, etc. & Consu...",1
00109.eda1664dd3b3c31b67e5cd04553b6546,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",1
00209.5276f967533f2ce0209c1eff631a86ff,This is a multi-part message in MIME format.\n...,1
00268.9b64189b6da55d1e0a30026ca73eb0da,HTML>\n\n<HEAD>\n\n<TITLE></TITLE>\n\n</HEAD>\...,1
